<h1><center>OMSF 2024: OpenFE Demo</center></h1>

<center><i><b>Exploring MCL-1 fragment RBFEs using OpenFE</b></i></center>
<br/><br/>

<h2><center>Tutorial Overview</center></h2>

This tutorial demonstrates how to set up and run a network of **Relative Binding Free Energy (RBFE)** calculations using the OpenFE toolkit.

Specifically we cover:
* Loading and defining systems
* Atom mapping and network creation
* Creating & running RBFE simulations
* Analyzing free energy results
* Using the Python & CLI interfaces

![rbfe_cycle](images/rbfe_thermocycle.png)

<h2><center>Test case: MCL-1 Fragments</center></h2>

* A set of **14 fragment** elaborations from a screen by *Friberg et al., J. Med Chem. 2013*.

* Part of the FEP+ fragment study by *Steinbrecher et al. J. Chem. Inf. Model 2015*

<center><img src="images/mcl1_frag_elaboration.png"/></center>

<center><img src="images/mcl1_site_crop.png"/></center>

<h2><center>Inspecting the Ligands</center></h2>

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

# Extract SDF contents, generate 2D coords (note: non-aligned scaffold!)
ligand_rdmols = [m for m in Chem.SDMolSupplier('ligands.sdf', removeHs=False)]
[AllChem.Compute2DCoords(ligand) for ligand in ligand_rdmols]

Chem.Draw.MolsToGridImage(ligand_rdmols, molsPerRow=5, subImgSize=(500,200))

<h2><center>The RBFE workflow</center></h2>

| **Step** | **External Tooling Used** |
|:---------|:----------------|
| 1. Creating OpenFE Components | RDKit, OpenMM, OpenFF |
| 2. Generating Atom Mappings | Kartograf, RDKit |
| 3. Creating a Ligand Network | Lomap, Networkx |
| 4. Defining a network of RBFE Alchemical Transformations | |
| 5. Running the simulations | OpenMM, OpenMMTools, Perses, OpenFF |
| 6. Gathering results | OpenMMTools, PyMBAR |
| 7. Analysis | Cinnabar, Matplotlib |

<center><img src="images/RBFE_overview.png"/></center>

<h2><center>OpenFE Components</center></h2>

In [ ]:
import openfe
from openff.units import unit

# ProteinComponent
protein = openfe.ProteinComponent.from_pdb_file(
    'protein.pdb'
)

# SolventComponent
solvent = openfe.SolventComponent(
    positive_ion='Na', negative_ion='Cl',
    neutralize=True,
    ion_concentration=0.15*unit.nanomolar
)

# Ligands
ligand_mols = [
    openfe.SmallMoleculeComponent(sdf)
    for sdf in Chem.SDMolSupplier(
        'ligands.sdf', removeHs=False
    )
]

<center><img src="images/OpenFEComponents.png"/></center>

<h2><center>Mapping Ligands</center></h2>

An atom mapping defines the atoms being mutated (and how) during the alchemical transformation.

* Uncoloured atoms: "mapped same elements"
* Blue atoms: "mapped different elements"
* Red atoms: "endstate dummy atoms"

In OpenFE we currently have two atom mappers:
1. `LomapAtomMapper` (MCS-based)

2. `KartografAtomMapper` (Geometry-based)

In [ ]:
from kartograf import KartografAtomMapper

mapper = KartografAtomMapper(atom_map_hydrogens=False)

atom_mapping = next(
    mapper.suggest_mappings(
        ligand_mols[0], ligand_mols[-1]
    )
)

atom_mapping

<h2><center>Mapping Ligands</center></h2>

We can also visualize the mappings in 3D!

*Note: atoms that have the same sphere color in both end states are mapped (i.e. will be transformed into dummy atoms in the opposite end state), whilst those without spheres are unmaped!*

In [ ]:
from openfe.utils import visualization_3D

visualization_3D.view_mapping_3d(atom_mapping)

<h2><center>Ligand Transformation Networks</center></h2>

We can use a mapper and a mapping scorer (e.g. `default_lomap_score`) to plan transformation networks between ligands of interest.

OpenFE can create various networks, including:
* Minimum Spanning Tree networks (MST)
* LOMAP networks
* Radial networks
* Loading external networks (e.g. FEP+)
* And many more using Konnektor!

In [ ]:
from openfe.setup.ligand_network_planning import (
    generate_lomap_network,
    generate_minimal_spanning_network,
    generate_radial_network,
)
from openfe import lomap_scorers

# Creating a Lomap network
lomap_network = generate_lomap_network(
    molecules=ligand_mols,
    scorer=lomap_scorers.default_lomap_score,
    mappers=[KartografAtomMapper()]
)

# Creating an MST network
mst_network = generate_minimal_spanning_network(
    ligands=ligand_mols,
    scorer=lomap_scorers.default_lomap_score,
    mappers=[KartografAtomMapper()]
)

In [ ]:
# A network is a collecting of mappings

mst_edges = [edge for edge in mst_network.edges]

print("atom mapping: ", mst_edges[0].componentA_to_componentB)

mst_edges[0]

In [ ]:
# We can visualize the networks too!
# Note: small visual artifact bug to be fixed ;)

from openfe.utils.atommapping_network_plotting import plot_atommapping_network

plot_atommapping_network(lomap_network)

In [ ]:
plot_atommapping_network(mst_network)

<h2><center>Creating an RBFE cycle</center></h2>

For each edge in our ligand network we can define a set of simulations to allow us to recover the **binding free energy**.

To do this we need to define:
  - 2 solvent states
  - 2 complex states
  - A state transformation simulation protocol

![rbfe_cycle](images/rbfe_thermocycle.png)

<h2><center>Defining end states</center></h2>

* `ChemicalSystems` can be defined for each end state of the cycle.

* These define the `Components` making up each state.

* Free energy Protocols take multiple ChemicalSystems to define the transformation being simulated.

![rbfe_cycle](images/rbfe_thermocycle.png)

<h2><center>Defining end states</center></h2>

In [ ]:
# Selecting a specific edge
edge = mst_edges[0]

# Components are collated into ChemicalSystems
ligand_A_complex = openfe.ChemicalSystem(
    {'ligand': edge.componentA,
     'protein': protein,
     'solvent': solvent}
)

ligand_A_solvent = openfe.ChemicalSystem(
    {'ligand': edge.componentA,
     'solvent': solvent}
)

ligand_B_complex = openfe.ChemicalSystem(
    {'ligand': edge.componentB,
     'protein': protein,
     'solvent': solvent}
)

ligand_B_solvent = openfe.ChemicalSystem(
    {'ligand': edge.componentB,
     'solvent': solvent}
)

In [ ]:
# We can inspect the equality of these
# ChemicalSystems to identify differences

another_complex = openfe.ChemicalSystem(
    {'ligand': edge.componentA,
     'protein': protein,
     'solvent': solvent}
)

print(ligand_A_complex == another_complex)

print(ligand_A_complex == ligand_B_complex)


<h2><center>Creating a simulation Protocol</center></h2>

An OpenFE **Protocol** defines how a simulation will take place.

Here we use the `RelativeHybridTopologyProtocol`, based on Perses, which:
  * Uses a hybrid topology approach
  * Allows for equilibrium samplers; HREX, SAMS, and independent windows
  * Uses OpenMM as a simulation engine

In [ ]:
from openfe.protocols.openmm_rfe import (
    RelativeHybridTopologyProtocol
)

# Protocols require settings, each has a default set
settings = RelativeHybridTopologyProtocol.default_settings()

# We can get the FF version
print(settings.forcefield_settings.small_molecule_forcefield)

# We can set the FF version
settings.forcefield_settings.small_molecule_forcefield = 'openff-2.2'
print(settings.forcefield_settings.small_molecule_forcefield)

<h2><center>Creating a simulation Protocol</center></h2>

* **Protocol** objects are created from their settings and are immutable.

* **Protocol** objects define how to apply a Transformation to a set of input ChemicalSystems & atom mappings.

* **Protocol** objects can be re-used to define multiple simulations.

In [ ]:
# Creating a Protocol for our RBFE simulations

rbfe_protocol = RelativeHybridTopologyProtocol(
    settings=settings
)

<h2><center>Creating Transformations</center></h2>

With `ChemicalSystem`s and a `Protocol` defined, we can create a set of `Transformation`s for our RBFE cycle.

A `Transformation` contains everything necessary to run a single simulation. For our chosen RFE Protocol this requires:
  * Two `ChemicalSystem` defining the end states
  * An atom mapping between the transforming ligands
  * A `Protocol` object
  * A name (optional)

In [ ]:
transformation_complex = openfe.Transformation(
    stateA=ligand_A_complex,
    stateB=ligand_B_complex,
    mapping=edge,
    protocol=rbfe_protocol,
    name="A_to_B_transformation_complex"
)

transformation_solvent = openfe.Transformation(
    stateA=ligand_A_solvent,
    stateB=ligand_B_solvent,
    mapping=edge,
    protocol=rbfe_protocol,
    name="A_to_B_transformation_solvent"
)

<h2><center>Saving Transformations</center></h2>

`Transformation`s can then be saved to file (JSON) for execution at a later date.

Executing a `Transformation` (see later) will yield a free energy estimate for that leg of the thermodynamic cycle.

In [ ]:
import pathlib

out_dir = pathlib.Path("single_transform")
out_dir.mkdir(exist_ok=True)

transformation_complex.dump(
    out_dir / f"{transformation_complex.name}.json"
)

transformation_solvent.dump(
    out_dir / f"{transformation_solvent.name}.json"
)

<h2><center>Networks of Transformations (AlchemicalNetworks)</center></h2>

We can create `AlchemicalNetwork`s which contain all the `Transformation` for all the simulations necessary for a network of RBFE calculations.

In [ ]:
transformations = []
for mapping in mst_network.edges:
    for leg in ['solvent', 'complex']:
        # use the solvent and protein created above
        sysA_dict = {'ligand': mapping.componentA,
                     'solvent': solvent}
        sysB_dict = {'ligand': mapping.componentB,
                     'solvent': solvent}
        
        if leg == 'complex':
            sysA_dict['protein'] = protein
            sysB_dict['protein'] = protein
        
        sysA = openfe.ChemicalSystem(sysA_dict)
        sysB = openfe.ChemicalSystem(sysB_dict)
        
        name = (f"{leg}_{mapping.componentA.name}_"
                f"{mapping.componentA.name}")
        
        transformation = openfe.Transformation(
            stateA=sysA,
            stateB=sysB,
            mapping={'ligand': mapping},
            protocol=rbfe_protocol,
            name=name
        )
        transformations.append(transformation)

network = openfe.AlchemicalNetwork(transformations)

In [ ]:
# Similarly we can write out all the AlchemicalNetwork
# Transformations to disk

# first we create the directory
transformation_dir = pathlib.Path("networktransforms")
transformation_dir.mkdir(exist_ok=True)

# then we write out each transformation
for transformation in network.edges:
    transformation.dump(transformation_dir / f"{transformation.name}.json")

In [ ]:
!ls networktransforms

<h2><center>Using the CLI instead</center></h2>

You can instead do all of this using a single line of the CLI!

The CLI can be quite convenient when you want to use default options. We are adding the ability to customize things using an optionall YAML file. For now this is limited to selecting atom mapper and network options.

*Example YAML settings*:
```yaml
mapper:
  method: kartograf
  
network:
  method: generate_minimal_spanning_network
```

In [ ]:
!openfe plan-rbfe-network -M ligands.sdf -p protein.pdb -o cli_setup -s settings.yaml

<h2><center>Running simulations</center></h2>

We can run each leg of our simulation using the `quickrun` command.

This takes one of the JSON files we wrote, runs the simulation, and writes an output JSON file with the simulation results.

```
openfe quickrun path/to/transformation.json -o results.json -d working-directory
```

You can loop over a list of JSON files to run through a network, or distribute your `quickrun` commands over a HPC cluster.

**Note:** alternative execution engines can also be used, such as [Alchemiscale](https://github.com/openforcefield/alchemiscale).

<h2><center>Gathering results</center></h2>

Once complete, the `quickrun` command will create a results JSON file and an accompanying directory with relevant simulation files.

In [ ]:
!ls results_jsons/easy_rbfe_ligand_12_solvent_ligand_13_solvent/shared*

<h2><center>Gathering results</center></h2>

These files include various structure (`.pdb`) and trajectory (`.nc`) files, and outputs from automated analyses.

For example we can find a PNG of the MBAR overlap matrix:

<center><img src="results_jsons/easy_rbfe_ligand_12_solvent_ligand_13_solvent/shared_RelativeHybridTopologyProtocolUnit-be5ac08159a948f58c5b40195b7c777c_attempt_0/mbar_overlap_matrix.png"/></center>

<h2><center>Gathering results</center></h2>

We can use the `gather` method to extract results directly from our simulations.

Here we report the relative binding free energies for each ligand pair, where the estimates and uncertainty are the average and standard deviation of three independent repeat.

In [ ]:
!openfe gather results_jsons/ --report ddg

<h2><center>Gathering results</center></h2>

There is also an option to get the absolute free energies using Maximum Likelihood Estimation with the [cinnabar](https://github.com/OpenFreeEnergy/cinnabar) toolkit.

In [ ]:
!openfe gather results_jsons/ --report dg

<h2><center>Gathering results</center></h2>

With a little bit of data wrangling with experimental results, and the use of the [cinnabar](https://github.com/OpenFreeEnergy/cinnabar) toolkit, we can plot these results out.

By comparison, the 2015 Steinbrecher et al. study yielded very similar results with within error RMSE and R2 values.

<center><img src="mcl1_exp_DDG.png"/></center>

<center><img src="mcl1_exp_DG.png"/></center>